1. 模型部署, c++ ?
2. 模型蒸馏？

先介绍一个模型架构查看工具：[Netron](https://github.com/lutzroeder/netron), 支持查看目前主流的DL框架导出的模型文件。


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.onnx
import onnx


# 定义模型
class IrisModel(nn.Module):
    def __init__(self, num_features, num_labels):
        super(IrisModel, self).__init__()
        self.fc_0 = nn.Linear(num_features, 25) 
        self.fc_1 = nn.Linear(25, 30) 
        self.fc_2 = nn.Linear(30, num_labels) 

    def forward(self, x):
        out = F.relu(self.fc_0(x))
        out = F.relu(self.fc_1(out))
        out = self.fc_2(out)
        return out

model = IrisModel(4,3) #模型初始化,特征的维度=4，iris有3个分类
example_inputs = torch.rand(2,4) #定义一个模型的输入数据，随机的就成
# print(example_inputs)
# print(example_inputs.shape)

In [2]:
# 1.使用torch.jit.trace保存模型文件
# https://pytorch.org/tutorials/advanced/cpp_export.html
traced_script_module = torch.jit.trace(model, example_inputs) #使用torch.jit.trace 模型
traced_script_module.save("traced_script_module.pt") #保存模型，可以使用Netron查看模型结构

# 使用c++ 加载模型文件
# 请求c++ 服务 

In [3]:
# 2.使用onnx格式保存模型文件 https://onnxruntime.ai/
# https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html
torch.onnx.export(model,             # model being run
           example_inputs,           # model input (or a tuple for multiple inputs)
          "traced_script_module.onnx",   # where to save the model (can be a file or file-like object)
          export_params=True,        # store the trained parameter weights inside the model file
          opset_version=10,          # the ONNX version to export the model to
          do_constant_folding=True,  # whether to execute constant folding for optimization
          input_names = ['input'],   # the model's input names
          output_names = ['output'], # the model's output names
          dynamic_axes={'input' : {0:'batch_size', 1:"num_features"},    # variable length axes
                        'output': {0:'batch_size', 1:"num_labels"}})

onnx_model = onnx.load("traced_script_module.onnx")
onnx.checker.check_model(onnx_model)